In [6]:
import os
import cv2
import numpy as np

sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [8]:
orginal_video_source = cv2.VideoCapture("videos/stereo/left_output.avi")
original_train_model = cv2.imread("images/final_exam/Templates/Template-1.png")
grayscale_train_model = cv2.cvtColor(original_train_model, cv2.COLOR_BGR2GRAY)

x_coordinate, y_coordinate, z_coordinate = 0, 0, 0
float_width, float_height = [float(sizes) for sizes in [0, 0]]

video_capture = orginal_video_source
if video_capture.isOpened():
    float_width = video_capture.get(3)
    float_height = video_capture.get(4)

divide_width, divide_height = [float(divide) for divide in [float_width / 2, float_height / 2]]

In [9]:
def objectDetection(train_image, grayscale_train_image, query_image, grayscale_query_image):
    train_image_keypoints, train_image_descriptors = sift.detectAndCompute(grayscale_train_image, None)
    query_image_keypoints, query_image_descriptors = sift.detectAndCompute(grayscale_query_image, None)

    matches = bf.knnMatch(train_image_descriptors, query_image_descriptors, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
            good_matches_list.append([m])

    if len(good_matches) > 10:
        src_pts = np.float32([ train_image_keypoints[m.queryIdx].pt for m in good_matches ]).reshape(-1, 1, 2)
        dst_pts = np.float32([ query_image_keypoints[m.trainIdx].pt for m in good_matches ]).reshape(-1, 1, 2)

        HOMOGRAPHY, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 0.7)

        h, w = train_image.shape[:2]
        train_image_box = np.float32([
            [0, 0], [0, h-1], [w-1, h-1], [w-1, 0]
        ]).reshape(-1, 1, 2)
        train_image_transformed_box = cv2.perspectiveTransform(train_image_box, HOMOGRAPHY)

        detected_image = cv2.polylines(query_image, [np.int32(train_image_transformed_box)], True, (255, 0, 255), 3, cv2.LINE_AA)
        x_coordinate = train_image_transformed_box[0][0]
        y_coordinate = train_image_transformed_box[1][0]

        first_popric = train_image_transformed_box[0]
        second_popric = train_image_transformed_box[1]
        total_popric = second_popric[0] - first_popric[0]

        cv2.putText(query_image, "x: {}, y: {}, z: {}".format(int(x_coordinate[0] - divide_width), int(y_coordinate[1] - divide_height), int(total_popric[1])), (50, 60), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

        return detected_image
    else:
        print("Keypoints not enough ...")
        return query_image

In [10]:
while video_capture.isOpened():
    ret, query_image = video_capture.read()
    grayscale_query_image = cv2.cvtColor(query_image, cv2.COLOR_BGR2GRAY)
    if ret:
        cv2.imshow("Object", objectDetection(original_train_model, grayscale_train_model, query_image, grayscale_query_image))
        if cv2.waitKey(int(1000 / 30)) & 0xFF == 27:
            break

video_capture.release()
cv2.destroyAllWindows()